In [1]:
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from datetime import datetime
import joblib
import os
import sys
sys.path.append(os.path.abspath('..'))
from download import local_download

In [18]:
# os.path.abspath('..')

'/home/daniel/programming/NLP/sentiment-hate-system/src'

In [2]:
EXPERIMENT_NAME = "sentiment-hate-experiment"
TRACKING_URI = 'http://127.0.0.1:5000'

mlflow.set_tracking_uri(TRACKING_URI)
client = MlflowClient()
experiment_id = client.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

print(mlflow.get_tracking_uri())
print(experiment_id)

http://127.0.0.1:5000
2


In [3]:
client.search_experiments()

[<Experiment: artifact_location='./artifacts/2', creation_time=1665562946949, experiment_id='2', last_update_time=1665562946949, lifecycle_stage='active', name='sentiment-hate-experiment', tags={}>,
 <Experiment: artifact_location='./artifacts/0', creation_time=None, experiment_id='0', last_update_time=None, lifecycle_stage='active', name='Default', tags={}>]

# List all logged models

In [4]:
runs = client.search_runs(
    experiment_ids="2",
    filter_string="",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=4,
#     order_by=["metrics.test-auc-score ASC"]
)

for run in runs:
    print(f"run id: {run.info.run_id}, misclass: {float(run.data.params['percent-misclassified']):.4f}, test auc: {run.data.metrics['test-auc-score']:.4f} train auc: {run.data.metrics['train-auc-score']:.4f}")

run id: 3032198944464d1d99afb894f020b9a5, misclass: 5.4542, test auc: 0.9830 train auc: 0.9968
run id: c46b3478a7a947668cbb849f52c2b89c, misclass: 5.3661, test auc: 0.9830 train auc: 0.9968
run id: bb62c33eeee04474b8b389e38734e37f, misclass: 5.3837, test auc: 0.9830 train auc: 0.9968
run id: 3bff3d4e681b4574872d26ecb645173a, misclass: 5.2251, test auc: 0.9831 train auc: 0.9972


In [5]:
client.search_registered_models()

[]

In [6]:
client.list_run_infos(experiment_id=experiment_id)[0].run_id

/tmp/ipykernel_9612/540392019.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.list_run_infos`` is deprecated. This method will be removed in a future release. Use ``search_runs()`` instead.
  client.list_run_infos(experiment_id=experiment_id)[0].run_id


'3032198944464d1d99afb894f020b9a5'

# Register Model

In [5]:
# Register Model
run_id = "3bff3d4e681b4574872d26ecb645173a"
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="sentiment-hate-classifier")

In [6]:
client.search_registered_models()

[<RegisteredModel: creation_timestamp=1665704332409, description=("This model detects the hate in a twitter's text. The data consists of a "
  'feature: Tweet.'), last_updated_timestamp=1665716817535, latest_versions=[<ModelVersion: creation_timestamp=1665706048896, current_stage='None', description='', last_updated_timestamp=1665706048896, name='sentiment-hate-classifier', run_id='bb62c33eeee04474b8b389e38734e37f', run_link='', source='./artifacts/2/bb62c33eeee04474b8b389e38734e37f/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='3'>,
  <ModelVersion: creation_timestamp=1665706033511, current_stage='Staging', description='The model version 2 was transitioned to Staging on 2022-10-13', last_updated_timestamp=1665708973557, name='sentiment-hate-classifier', run_id='c46b3478a7a947668cbb849f52c2b89c', run_link='', source='./artifacts/2/c46b3478a7a947668cbb849f52c2b89c/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>

# Update the description of Registered Model

In [6]:
client.update_registered_model(
  name="sentiment-hate-classifier",
  description="This model detects the hate in a twitter's text. The data consists of a feature: Tweet."
)

<RegisteredModel: creation_timestamp=1665704332409, description=("This model detects the hate in a twitter's text. The data consists of a "
 'feature: Tweet.'), last_updated_timestamp=1665705096730, latest_versions=[<ModelVersion: creation_timestamp=1665704332492, current_stage='None', description='', last_updated_timestamp=1665704332492, name='sentiment-hate-classifier', run_id='3032198944464d1d99afb894f020b9a5', run_link='', source='./artifacts/2/3032198944464d1d99afb894f020b9a5/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='sentiment-hate-classifier', tags={}>

In [24]:
latest_versions = client.get_latest_versions(name="sentiment-hate-classifier")
print(f"version: {latest_versions[0].version}, stage: {latest_versions[0].current_stage}")

version: 4, stage: None


In [27]:
for v in latest_versions:
    print(f"version: {v.version}, stage: {v.current_stage}")

version: 4, stage: None


# Transition model to Staging

In [36]:
model_name="sentiment-hate-classifier"
model_version=4
# Transition model version stage
client.transition_model_version_stage(name=model_name,
                                      version=model_version,
                                      stage='Staging',
                                      archive_existing_versions=False)

<ModelVersion: creation_timestamp=1665706066384, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2022-10-13', last_updated_timestamp=1665708980147, name='sentiment-hate-classifier', run_id='3bff3d4e681b4574872d26ecb645173a', run_link='', source='./artifacts/2/3bff3d4e681b4574872d26ecb645173a/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='4'>

In [37]:
client.update_model_version(name=model_name,
                            version=model_version,
                            description=f"The model version {model_version} was transitioned to Staging on {datetime.today().date()}")

<ModelVersion: creation_timestamp=1665706066384, current_stage='Staging', description='The model version 4 was transitioned to Staging on 2022-10-13', last_updated_timestamp=1665708982214, name='sentiment-hate-classifier', run_id='3bff3d4e681b4574872d26ecb645173a', run_link='', source='./artifacts/2/3bff3d4e681b4574872d26ecb645173a/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='4'>

In [38]:
client.transition_model_version_stage(name=model_name,
                                      version=model_version,
                                      stage='Production',
                                      archive_existing_versions=False)
client.update_model_version(name=model_name,
                            version=model_version,
                            description=f"The model version {model_version} was moved to Production on {datetime.today().date()}")

<ModelVersion: creation_timestamp=1665706066384, current_stage='Production', description='The model version 4 was moved to Production on 2022-10-13', last_updated_timestamp=1665716817619, name='sentiment-hate-classifier', run_id='3bff3d4e681b4574872d26ecb645173a', run_link='', source='./artifacts/2/3bff3d4e681b4574872d26ecb645173a/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='4'>

In [3]:
# Load model model in production
run_id = "3bff3d4e681b4574872d26ecb645173a"
print(model_uri)

In [6]:
model_uri

'runs:/3bff3d4e681b4574872d26ecb645173a/models'

In [6]:
model = mlflow.lightgbm.load_model(model_uri)

In [7]:
model

In [5]:
# Download registed model artifacts in your local machine
run_id="3bff3d4e681b4574872d26ecb645173a"
model_uri=f"runs:/{run_id}/models"

local_download.main(run_id=run_id)

Model with run id: 3bff3d4e681b4574872d26ecb645173a has been successfully downloaded.


In [10]:
print(os.path.realpath(f'../artifacts'))

/home/daniel/programming/NLP/sentiment-hate-system/src/artifacts


In [7]:
# client.download_artifacts(run_id=run_id, path="", dst_path=os.path.realpath('../artifacts/2/3bff3d4e681b4574872d26ecb645173a'))

'/home/daniel/programming/NLP/sentiment-hate-system/src/artifacts/2/3bff3d4e681b4574872d26ecb645173a/'

In [ ]:
"./artifacts/2/3bff3d4e681b4574872d26ecb645173a/artifacts/models"

In [10]:
client.download_artifacts(run_id=run_id, path="tfVectorizer", dst_path=os.path.realpath('../vectorizers'))

'/home/daniel/programming/NLP/sentiment-hate-system/src/vectorizers/tfVectorizer'

In [13]:
vectorizer = joblib.load(open(os.path.join(os.path.realpath('../vectorizers'),f"tfVectorizer/vectorizerV7.bin"), "rb"))

In [12]:
os.path.realpath('../vectorizers')

'/home/daniel/programming/NLP/sentiment-hate-system/src/vectorizers'

In [14]:
vectorizer

TfidfVectorizer(encoding='latin-1', max_features=1000, min_df=5,
                ngram_range=(1, 2), sublinear_tf=True)